# Google Earth Engine
Alexander Bedine  
ajb20dd@fsu.edu  
November 1, 2024

## Table of Contents

 - [Environment Setup](#environment-setup)
 - [Code](#code)

### Environment Setup

In [1]:
import ee
import geemap
import geopandas as gpd

### Code

In [2]:
Map = geemap.Map(center=(40, -100), zoom=3)

USGS_3DEP_10m = ee.Image("USGS/3DEP/10m")
elevation = USGS_3DEP_10m.select('elevation')
viz = {
    'min': 0,
    'max': 3000,
    'palette': ['3ae237', 'b5e22e', 'd6e21f', 'fff705', 'ffd611', 'ffb613', 'ff8b13',
    'ff6e08', 'ff500d', 'ff0000', 'de0101', 'c21301', '0602ff', '235cb1',
    '307ef3', '269db1', '30c8e2', '32d3ef', '3be285', '3ff38f', '86e26f']
}

Map.addLayer(elevation, viz, 'elevation')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

Create a Map using the USGS DEM that shows the elevation of the United States.

In [3]:
us_counties = ee.FeatureCollection("TIGER/2018/Counties")
Map.addLayer(us_counties, {'opacity': 0.7}, "US Counties")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

add the boundaries for US Counties based on the 2018 Census to the map.

In [4]:
import us
from census import Census

In [5]:
path = 'https://www2.census.gov/geo/tiger/TIGER2018/COUNTY/tl_2018_us_county.zip'

census_gdf = gpd.read_file(path)

In [6]:
features = geemap.geopandas_to_ee(census_gdf)

using the shape files from TIGER to define a **features** variable, since doing it otheriwse created issues where it was too much data to be processed.

In [15]:
geemap.zonal_stats(elevation, features, "elevresults.geojson", stat_type="MAXIMUM")

Computing statistics ...


wow I should've ran this on my desktop